Implementation of Algorithm SCS, presented on pages 42-43.

In [ ]:
#Block 1: loading packages
import numpy as np
import scipy as sc
import random
from scipy import linalg as la
from numpy.linalg import norm
import scipy.sparse as sparse
from scipy.sparse import rand as rndma
import time

In [ ]:
#Block 2: setting the rounding and precision parameters
prec = 8 #rounding parameter
Eps = 10**(-prec)

In [ ]:
#Block 3: function which generates a row of a starting matrix, positive case
def vecgen_poz():
    vec = np.random.rand(dim,)
    vec = np.round(vec,prec)
    return vec

In [ ]:
#Block 4: function which generates a row of a starting matrix, sparse case
def vecgen_sparse():
    ranrow = lambda: (rndma(1,dim,density = np.random.uniform(a,b))).A
    v = ranrow()
    v = v.flatten()
    v = np.round(v,prec)
       
    return v

In [ ]:
#Block 5: selective power method 
def pwrmthd(A):
    A = A + np.identity(dim) #see the remark on page 28
    v0 = np.array([1 for i in xrange(dim)]) #starting vector of all ones 
    v1 = np.dot(A,v0)/float(norm(np.dot(A,v0)))
    v1 = np.round(v1,prec)
    while norm(v0-v1) > Eps*10: #the precision parametar $\varepsilon$ 
        v0 = v1
        v1 = np.dot(A,v0)/float(norm(np.dot(A,v0)))
    return np.round(v1,prec-1)

In [ ]:
#Block 6: computing the spectral radius
def leading(A):
    
    evals = np.linalg.eig(A)[0] #set of eigenvalues 
    evals = np.round(evals,prec)
    return np.amax(np.real(evals)) #spectral radius

In [ ]:
#Block 7: a solution to the LP problem
def lp_solution(A,v,supp,tau):
    
    D = len(supp)
    X = np.copy(A)
    
    ind = np.argsort(v)[::-1]
    ind = ind[:D]
    
    for i in xrange(dim):
        S = 0
        for l in ind:
            S += A[i,l]
            if (S <= tau):
                X[i,l] = 0
            else:
                X[i,l] = -tau + S
                break
    
    return np.round(X,prec)

In [ ]:
#Block 8: implementing the the greedy method for minimization on the ball 
#of radius \tau (Step 1)
def selective_greedylinf(A,tau):
    
    X = np.copy(A)
    v0 = pwrmthd(X) #computing the leading eigenvalue
    supp = list(np.where(v0 != 0)[0]) #getting the support
    notsupp = list(set(range(dim)) - set(supp))
    notsupp.sort()

    while True: #constructing the solution X_k
        Z = np.copy(X)
        v = v0
        X = lp_solution(A,v,supp,tau)
        X[notsupp] = Z[notsupp]
        
        for k in supp:
            olddot = np.dot(Z[k],v)
            newdot = np.dot(X[k],v)
            if (olddot < newdot) or (abs(olddot - newdot) < 1e-6): #see the Appendix
                X[k] = Z[k]
        
        v0 = pwrmthd(X)
        spect_radius = leading(X)
        
        '''if matrices of iterations k-1 and k match on the support, 
        OR if they have the same leading eigenvector,
        OR if the spectral radius of X_k is less than 1 - \theta, 
        we finish the greedy method'''
        if (X[supp] == Z[supp]).all() or (v == v0).all() or (spect_radius < 1 - theta):
            supp = list(np.where(v0 != 0)[0])
            return np.round(X,prec), spect_radius, v0, supp
        else:
            supp = list(np.where(v0 != 0)[0])
            notsupp = list(set(range(dim)) - set(supp))
            notsupp.sort()

In [ ]:
#Block 9: the greedy method for checking if the potential solution is true solution (Step 4)
def selective_greedylinf_check(A,tau):
    
    X = np.copy(A)
    v0 = pwrmthd(X) #computing the leading eigenvalue
    supp = list(np.where(v0 != 0)[0]) #getting the support
    notsupp = list(set(range(dim)) - set(supp))
    notsupp.sort()

    while True: #constructing the solution X_k
        Z = np.copy(X)
        v = v0
        X = lp_solution(A,v,supp,tau)
        X[notsupp] = Z[notsupp]
        
        for k in supp:
            olddot = np.dot(Z[k],v)
            newdot = np.dot(X[k],v)
            if (olddot < newdot) or (abs(olddot - newdot) < 1e-6): #see page X
                X[k] = Z[k]
        
        v0 = pwrmthd(X)
        spect_radius = leading(X)
        
        '''if matrices of iterations k-1 and k match on the support, 
        OR if they have the same leading eigenvector, we finish the greedy method''' 
        if (X[supp] == Z[supp]).all() or (v == v0).all():
            return np.round(X,prec), spect_radius
        
            '''if we obtain a stable matrix with a spect. radius too far from one,
            we quit the procedure; our matrix is not an optimal one'''
        elif (spect_radius < 1) and (abs(spect_radius - 1) >= theta):
            return _,1000
        
        else:
            supp = list(np.where(v0 != 0)[0])
            notsupp = list(set(range(dim)) - set(supp))
            notsupp.sort()

In [ ]:
#Block 10: construcitng the matrices R and C (Step 2) 
def costruct_RC(A,v,tau): 
    D = len(v)
    C = np.copy(A)
    R = np.zeros((D,D))
    tausum = []
    
    ind = np.argsort(v)[::-1] #reordering
    
    for i in xrange(D): #constructing the solution
        S = 0
        for l in ind:
            S += A[i,l]
            if (S <= tau):
                C[i,l] = 0
            else:
                C[i,l] = S
                R[i,l] = 1
                break
    
    return R, C

In [ ]:
#Block 11: finding the closest Schur stable matrix
def schur_stabilization(A): 
    
    start = time.clock() #time counter
     
    tau0 = norm(A,np.Inf)/2.0 #the starting \tau (Step 0)
    seg = tau0
    tau1 = 0
    
    while True:
        while True: #this loop is the bisection in tau (Step 1)
            X, spect_radius, v, supp = selective_greedylinf(A,tau0)
            print tau0, spect_radius
    
           
            taudiff = abs(tau1 - tau0)
            spectdiff = abs(spect_radius - 1)
        
            if (spectdiff < theta) and (taudiff < 1e-3):
                '''\theta is the tolerance parameter; if we arrive by bisection at the 
                matrix having X_k with spect. rad. 1 +- \theta, we will accept it as 
                a solution;
                taudiff is put for precaution: we can accept the matrix X_k as the solution
                only if \tau_{k-1} and \tau_k are close enough, else we might get an
                imprecize solution'''
                run = time.clock() - start
                run = np.round(run,2)
                return run, spect_radius, norm(X-A,np.Inf)

            if (spect_radius > 1):
                tau1 = tau0
                seg /= 2.0
                tau0 += seg

            else: #we go to the next step if we obtain a stable matrix
                tau1 = tau0
                seg /= 2.0
                tau0 -= seg
                break
                
               
        # Here we construct matrices R and C (Step 2)        
        Aredux = A[np.ix_(supp,supp)]
        vredux = v[supp]
        C = np.copy(X)
        R = np.zeros((dim,dim))
        Rredux, Credux = costruct_RC(Aredux,vredux,tau1)
        R[np.ix_(supp,supp)] = Rredux
        C[np.ix_(supp,supp)] = Credux
        Sprim = np.identity(dim) - (C - tau1*R) #the matrix given on page 42
        det = np.linalg.det(Sprim)
        #computing the potential optimal value (Step 3)
        if (det != 0): 
        
        
            S = la.inv(Sprim)
            S = np.round(S,prec)
            Q = np.matmul(S,R)
            lamb = leading(Q)
            taustar = tau1 - 1/lamb
            taustar = np.round(taustar,prec) #the potential optimal distance


            Xstar, spect_final = selective_greedylinf_check(A,taustar)
            #checking if we really have a true solution (Step 4)
            if (abs(spect_final - 1) < theta): 
                '''checking if the obtained matrix has a spectral radius in 
                accordance to the tolerance parameter'''
                run = time.clock() - start
                run = np.round(run,2)
                return run, spect_final, norm(Xstar-A,np.Inf)

In [ ]:
#Block 12: running the algorithm SCS, sparse case
dim = 600 #set the dimension
theta = 1e-4 #set the tolerance parameter
(a,b) = (0.09,0.16) #set the interval for the density parameter
A = np.array([vecgen_sparse() for k in xrange(dim)], dtype = float) #the starting matrix
S = schur_stabilization(A)
print S